In [1]:
import copy
from tqdm.auto import tqdm
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


from superprimitive_fusion.scanner import (
    capture_spherical_scans,
    virtual_mesh_scan,
    mesh_depth_image,
    generate_rgbd_noise,
    clean_mesh_and_remap_weights,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
    distinct_colours,
)
from superprimitive_fusion.mesh_fusion import (
    fuse_meshes,
)
from superprimitive_fusion.mesh_fusion_utils import (
    get_mesh_components,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    # ('table', 'table.obj'),
    # ('power-drill', 'power-drill.obj'),
    # ('bleach', 'bleach.obj'),
    # ('pitcher', 'pitcher.obj'),
    # ('mug', 'mug.obj'),
    # ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

gt_meshes = dict()
for foldername,filename in names:
    print(f'Getting the {foldername}')
    
    gt_mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(gt_mesh)
    gt_mesh.compute_vertex_normals()

    gt_meshes[foldername] = gt_mesh

gt_mesh_list = list(gt_meshes.values())

Getting the mustard-bottle


In [3]:
centres = []
for gt_meshname, gt_mesh in gt_meshes.items():
    if gt_meshname == 'table':
        continue
    centres.append(gt_mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [15]:
cam_offset=obj_centre
look_at = obj_centre
width: int = 360
height: int = 240
fov: float = 70.0
k: float = 10
max_normal_angle_deg = None
N = 6
radius = 0.25
include_depth_images = True

obj_scans = capture_spherical_scans(
    gt_mesh_list,
    N,
    radius,
    look_at,
    cam_offset,
    width,
    height,
    fov,
    k,
    max_normal_angle_deg,
    "latlong",
    constant_uncertainty    =2e-4,
    linear_uncertainty      =2e-3,
    quadrt_uncertainty      =1e-4,
    constant_perlin_sigma   =2e-4,
    linear_perlin_sigma     =2e-3,
    quadrt_perlin_sigma     =1e-4,
    perlin_octaves          =3,
    seg_scale_std           =1e-4,
    rot_std                 =1e-4,
    trn_std                 =1e-3,
    grazing_lambda          =1.0,
    include_depth_images=include_depth_images,
)

scan_mw_pairs = [(obj_scan['mesh'][0][0],obj_scan['mesh'][1][0]) for obj_scan in obj_scans]
if include_depth_images:
    depth_data = [obj_scan['mesh'][2] for obj_scan in obj_scans]
scan_meshes = [pair[0] for pair in scan_mw_pairs]

In [16]:
for mesh in scan_meshes:
    mesh.compute_vertex_normals()
o3d.visualization.draw_geometries(scan_meshes)

In [24]:
import numpy as np
import open3d as o3d

def make_intrinsic_from_K(K, w, h):
    K = np.asarray(K, float)
    return o3d.camera.PinholeCameraIntrinsic(w, h, K[0,0], K[1,1], K[0,2], K[1,2])

def _o3d_color(rgb):
    arr = np.asarray(rgb)
    if arr.dtype.kind in "fc":
        arr = (np.clip(arr, 0, 1) * 255).round().astype(np.uint8)
    else:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    return o3d.geometry.Image(np.ascontiguousarray(arr))

def _o3d_depth(depth, meters=True):
    d = np.asarray(depth)
    d[~np.isfinite(d)] = 0
    if meters:
        return o3d.geometry.Image(d.astype(np.float32)), 1.0
    else:
        return o3d.geometry.Image(d.astype(np.uint16)), 1000.0

def tsdf_fuse(
    depth_images, rgb_images, K, E_wc_list,
    voxel_length=1.0/1024.0, trunc_voxels=4, depth_trunc=1.0,
    depth_in_meters=True
):
    """Extrinsics MUST be world->camera (E_wc)."""
    assert len(depth_images)==len(rgb_images)==len(E_wc_list)
    h, w = depth_images[0].shape[:2]
    intr = make_intrinsic_from_K(K, w, h)
    sdf_trunc = trunc_voxels * voxel_length

    vol = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=voxel_length,
        sdf_trunc=sdf_trunc,
        color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8,
    )

    for depth, rgb, E_wc in zip(depth_images, rgb_images, E_wc_list):
        dimg, depth_scale = _o3d_depth(depth, meters=depth_in_meters)
        cimg = _o3d_color(rgb)
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            cimg, dimg, depth_scale=depth_scale, depth_trunc=depth_trunc, convert_rgb_to_intensity=False
        )
        vol.integrate(rgbd, intr, np.asarray(E_wc, dtype=np.float64))  # pass world->camera directly

    mesh = vol.extract_triangle_mesh()
    mesh.compute_vertex_normals()
    return mesh

depth_images = [d['depth'] for d in depth_data]
for d in depth_images:
    d[~np.isfinite(d)] = 0
rgb_images   = [d['rgb'] for d in depth_data]
K            = depth_data[0]['K_t']

T_wc = [d['E'] for d in depth_data]

mesh = tsdf_fuse(
    depth_images, rgb_images, K, T_wc,
    voxel_length=1./1024,
    trunc_voxels=5,
)
o3d.visualization.draw_geometries([mesh])

In [18]:
mesh = mesh.filter_smooth_taubin(
    number_of_iterations=50,
)
mesh.compute_vertex_normals()

TriangleMesh with 67579 points and 133351 triangles.

In [19]:
o3d.visualization.draw_geometries([mesh])

In [20]:
diff_mesh_colours = distinct_colours(len(scan_meshes))
diff_col_meshes = []
for i,mesh in enumerate(scan_meshes):
    if len(np.asarray(mesh.vertices)) == 0:
        continue
    col_mesh = copy.deepcopy(mesh)
    col_mesh.compute_vertex_normals()
    col_mesh.paint_uniform_color(diff_mesh_colours[i])
    diff_col_meshes.append(col_mesh)


front = look_at - (cam_offset + polar2cartesian(0.8, 110, 90))
front /= np.linalg.norm(front)
o3d.visualization.draw_geometries(
    geometry_list=diff_col_meshes,
    lookat=look_at,
    front=front,
    up=np.array([0, 0, 1]),
    zoom=0.7,
)

In [22]:
assert isinstance(scan_mw_pairs, list) and len(scan_mw_pairs) > 0, "scan_mw_pairs must be a non-empty list"
for i, (m, w) in enumerate(scan_mw_pairs):
    assert w.shape[0] == len(np.asarray(m.vertices)), f"weights length mismatch at pair {i}"

fused_mesh, fused_weights = scan_mw_pairs[0]

for t, (mesh, weights) in enumerate(tqdm(scan_mw_pairs[1:], desc="Scan", unit="scan"), start=1):
    fused_mesh, fused_weights = fuse_meshes(
        mesh1=fused_mesh, weights1=fused_weights,
        mesh2=mesh,       weights2=weights,
        h_alpha=5.0, r_alpha=2.0,
        nrm_shift_iters=3, nrm_smth_iters=1,
        shift_all=False,
    )

Scan:   0%|          | 0/5 [00:00<?, ?scan/s]

In [23]:
front = look_at - (cam_offset + polar2cartesian(0.8, 110, 90))
front /= np.linalg.norm(front)
o3d.visualization.draw_geometries(
    geometry_list=[fused_mesh],
    lookat=look_at,
    front=front,
    up=np.array([0, 0, 1]),
    zoom=0.7,
)

In [106]:
import os, tempfile
from pathlib import Path
import numpy as np
import pymeshlab as ml

# Optional: only if you pass Open3D meshes directly
try:
    import open3d as o3d
    _HAS_O3D = True
except Exception:
    _HAS_O3D = False

def _is_o3d_mesh(x):
    return _HAS_O3D and isinstance(x, o3d.geometry.TriangleMesh)

def _ensure_path(obj, suffix=".obj"):
    if isinstance(obj, (str, Path)):
        return str(obj), None
    if _is_o3d_mesh(obj):
        tmp = tempfile.NamedTemporaryFile(suffix=suffix, delete=False); tmp.close()
        ok = o3d.io.write_triangle_mesh(tmp.name, obj, write_ascii=False, compressed=False)
        if not ok: raise RuntimeError("Open3D write failed.")
        return tmp.name, tmp.name
    raise TypeError("gt/recon must be a filepath or open3d TriangleMesh")

def _cleanup_tmp(p):
    if p and os.path.isfile(p):
        try: os.remove(p)
        except OSError: pass

def _list_ids(ms):
    try: return list(ms.mesh_id_list())
    except Exception: return [i for i in range(ms.number_meshes())]

def _dist_array_from_mesh(ms, mesh_id):
    m = ms.mesh(mesh_id)
    if hasattr(m, "vertex_quality_array"):
        q = np.asarray(m.vertex_quality_array()).ravel()
        if q.size and np.any(np.isfinite(q)): return q
    if hasattr(m, "vertex_scalar_array"):
        s = np.asarray(m.vertex_scalar_array()).ravel()
        if s.size and np.any(np.isfinite(s)): return s
    raise RuntimeError("No per-vertex quality/scalar on mesh id=%s." % mesh_id)

def _summ(d):
    d = np.asarray(d); d = d[np.isfinite(d)]
    return dict(
        mean=float(d.mean()),
        median=float(np.median(d)),
        rms=float(np.sqrt((d**2).mean())),
        p95=float(np.quantile(d, 0.95)),
        p99=float(np.quantile(d, 0.99)),
        hausdorff=float(d.max()),
        trimmed_hausdorff_99=float(np.quantile(d, 0.99)),
        count=int(d.size),
    )

def _fscore(d_re2gt, d_gt2re, taus_abs):
    out = {}
    a = np.asarray(d_re2gt); b = np.asarray(d_gt2re)
    for t in taus_abs:
        P = float((a <= t).mean()) if a.size else 0.0   # how much of recon is within t of GT
        R = float((b <= t).mean()) if b.size else 0.0   # how much of GT is within t of recon
        F = 0.0 if (P+R)==0 else 2*P*R/(P+R)
        out[float(t)] = {"tau_m": float(t), "precision": P, "recall": R, "fscore": F}
    return out

def evaluate_pair_pml_meters(
    gt, recon,
    samples_per_mesh=200_000,
    taus_abs=(2.5e-4, 5e-4, 1e-3, 2e-3),   # 0.25, 0.5, 1, 2 mm
    include_topology=True,
    return_distance_arrays=False,
    debug=False
):
    gt_path, tmp_gt = _ensure_path(gt, suffix=".obj")
    re_path, tmp_re = _ensure_path(recon, suffix=".obj")
    try:
        ms = ml.MeshSet()
        ms.load_new_mesh(gt_path);    gt_id    = ms.current_mesh_id()
        ms.load_new_mesh(re_path);    recon_id = ms.current_mesh_id()

        # recon → GT  (savesample=True and capture the new layer id)
        before = set(_list_ids(ms))
        ms.set_current_mesh(recon_id)
        ms.apply_filter('get_hausdorff_distance',
                        targetmesh=gt_id,
                        samplevert=False, sampleedge=False, sampleface=True,
                        samplenum=samples_per_mesh, savesample=True)
        rg_sample_id = sorted(list(set(_list_ids(ms)) - before))[-1]
        d_re2gt = _dist_array_from_mesh(ms, rg_sample_id)

        # GT → recon
        before = set(_list_ids(ms))
        ms.set_current_mesh(gt_id)
        ms.apply_filter('get_hausdorff_distance',
                        targetmesh=recon_id,
                        samplevert=False, sampleedge=False, sampleface=True,
                        samplenum=samples_per_mesh, savesample=True)
        gr_sample_id = sorted(list(set(_list_ids(ms)) - before))[-1]
        d_gt2re = _dist_array_from_mesh(ms, gr_sample_id)

        if debug:
            print("re→gt min/med/max (m):", float(d_re2gt.min()), float(np.median(d_re2gt)), float(d_re2gt.max()))
            print("gt→re min/med/max (m):", float(d_gt2re.min()), float(np.median(d_gt2re)), float(d_gt2re.max()))
            print("sample sizes:", d_re2gt.size, d_gt2re.size)

        stats = {
            "recon_to_gt": _summ(d_re2gt),
            "gt_to_recon": _summ(d_gt2re),
            "fscore": _fscore(d_re2gt, d_gt2re, taus_abs=taus_abs),
        }

        if include_topology:
            ms.set_current_mesh(recon_id)
            stats["recon_topology"] = ms.apply_filter('get_topological_measures')
            ms.set_current_mesh(gt_id)
            stats["gt_topology"] = ms.apply_filter('get_topological_measures')

        if return_distance_arrays:
            stats["_distances"] = {"recon_to_gt": d_re2gt, "gt_to_recon": d_gt2re}

        return stats
    finally:
        _cleanup_tmp(tmp_gt); _cleanup_tmp(tmp_re)

In [108]:
res = evaluate_pair_pml_meters(gt_mesh, fused_mesh, samples_per_mesh=150_000, taus_abs=(2.5e-4, 5e-4, 1e-3, 2e-3), debug=True)
print(res)

[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.


/tmp/ipykernel_106824/2243590511.py:33: UserWarning: Deprecated Function Warning: use 'mesh_number()' instead.
  except Exception: return [i for i in range(ms.number_meshes())]


re→gt min/med/max (m): 3.1422425078586613e-10 5.818026643791829e-05 0.0009832752442970882
gt→re min/med/max (m): 2.781069263768643e-10 5.766877824046543e-05 0.0027824305719997442
sample sizes: 150000 150000
{'recon_to_gt': {'mean': 9.050021107892519e-05, 'median': 5.818026643791829e-05, 'rms': 0.0001338071768693322, 'p95': 0.0002887210183372055, 'p99': 0.00047102151224861306, 'hausdorff': 0.0009832752442970882, 'trimmed_hausdorff_99': 0.00047102151224861306, 'count': 150000}, 'gt_to_recon': {'mean': 9.193815261402374e-05, 'median': 5.766877824046543e-05, 'rms': 0.00014659031683699923, 'p95': 0.00029087849876854123, 'p99': 0.0005247199829456394, 'hausdorff': 0.0027824305719997442, 'trimmed_hausdorff_99': 0.0005247199829456394, 'count': 150000}, 'fscore': {0.00025: {'tau_m': 0.00025, 'precision': 0.9287133333333333, 'recall': 0.9289466666666667, 'fscore': 0.9288299853459608}, 0.0005: {'tau_m': 0.0005, 'precision': 0.9918533333333334, 'recall': 0.9884733333333333, 'fscore': 0.990160448859

In [111]:
o3d.visualization.draw_geometries([gt_mesh,fused_mesh])

In [114]:
print(res.keys())
print(res['recon_to_gt']['mean']*1000)

dict_keys(['recon_to_gt', 'gt_to_recon', 'fscore', 'recon_topology', 'gt_topology'])
0.09050021107892518
